In [1]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install langid
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 14.3 MB/s 
     |████████████████████████████████| 1.2 MB 40.5 MB/s 
     |████████████████████████████████| 596 kB 61.5 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 3.3 MB 45.6 MB/s 
     |████████████████████████████████| 880 kB 53.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=531e437f3894dc948775494a4127b654df4bc61fd8fd2997289eb57ea9ce87c4
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██

In [2]:
!sudo apt-get install hunspell
!sudo apt-get install hunspell-th

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  dictionaries-common emacsen-common hunspell-en-us libhunspell-1.6-0
  libtext-iconv-perl
Suggested packages:
  wordlist openoffice.org-hunspell | openoffice.org-core
The following NEW packages will be installed:
  dictionaries-common emacsen-common hunspell hunspell-en-us libhunspell-1.6-0
  libtext-iconv-perl
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 602 kB of archives.
After this operation, 2,276 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 emacsen-common all 2.0.8 [17.6 kB]
Get:3 http://archive.ubuntu.com/ubu

In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification
from tqdm import tqdm 
import pickle
from datasets import load_metric
import subprocess
import sys
import langid
from nltk.translate.gleu_score import sentence_gleu
from jiwer import cer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/tpth/tokenizer_40k_nova.pkl', 'rb'))

In [14]:
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/tpth/mlm_ds_40k_nova.pkl', 'rb'))
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/tpth/ner_ds_40k_nova.pkl', 'rb'))

In [15]:
ds_mlm = ds_mlm[37890:]
ds_tag = ds_tag[37890:]
NUM_SAMPLE = ds_mlm.shape[0]
NUM_SAMPLE

5000

In [10]:
def spell(word,lang):
    if sys.platform == 'win32':
        cmd = "echo "+word+" | hunspell -d "+lang
    else:
        cmd = 'echo "'+word+'" | hunspell -d '+lang
    getoutput = subprocess.getoutput(cmd)
    del cmd
    get = getoutput.split("\n")
    del get[0]
    if get[0] == '*':
        getoutput = "No Suggestions"
    else:
        if get[1] == "":
            del get[1]
        get = get[0].split(":")
        del get[0]
        getoutput = get[0].replace(" ","")
        getoutput = getoutput.split(",")
    del get
    return getoutput
Input = spell("appld","")
print(Input)
InputTH = spell("สถาปัตยกรรม","th_TH")
print(InputTH)
trueth = spell("กิร","th_TH")
print(trueth)

['apple', 'apply', 'applaud']
No Suggestions
['ก', 'กิริ']


In [16]:
bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0
total_gs_ori = 0
total_gs_pred = 0
total_cer_ori = 0
total_cer_pred = 0

# NUM_SAMPLE=1
for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_id = [k for k in text_id if k != 1]
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  labels_id = [k for k in labels_id if k != 1]
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]
  # print(text_id)
  # print(labels_id)

  # print(f"NO. {i}, {''.join(text_token)}")

  for j in range(text_token_len):
    text_token[j] = text_token[j].replace("▁", " ")
    if langid.classify(text_token[j])[0] == 'th':
      try:
        corrected = spell(text_token[j],"th_TH")
      except:
        print(f"err hunspell: {text_token[j]}")
      else:
        if not corrected == "No Suggestions":
          # print(f"{text_token[j]} => {corrected[0]}")
          text_token[j] = corrected[0]
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  # print(len(text_token), len(labels_token))
  # print(text_token)
  # print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  references[0][0] = [k.replace("▁", " ") for k in references[0][0]]
  original[0] = [k.replace("▁", " ") for k in original[0]]
  predictions[0] = [k.replace("▁", " ") for k in predictions[0]]

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(references)
  # print(predictions)
  # print(original)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  cer_text_ori = "".join(original[0])
  cer_text_pred = "".join(predictions[0])
  cer_text_ref = "".join(references[0][0])
  cer_text_ori = cer_text_ori.replace("_", "")
  cer_text_ori = cer_text_ori.replace("▁", "")
  cer_text_pred = cer_text_pred.replace("_", "")
  cer_text_pred = cer_text_pred.replace("▁", "")
  cer_text_ref = cer_text_ref.replace("_", "")
  cer_text_ref = cer_text_ref.replace("▁", "")
  # print(f"ORI: {cer_text_ori}")
  # print(f"PRED: {cer_text_pred}")
  # print(f"REF: {cer_text_ref}")
  cer_original = cer(cer_text_ori, cer_text_ref)
  total_cer_ori += cer_original
  cer_pred = cer(cer_text_pred, cer_text_ref)
  total_cer_pred += cer_pred

  # print(f"GLUE PREDICTED: {gleu_score}")
  # print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(f"CER PREDICTED: {cer_pred}")
  # print(f"CER ORIGINAL: {cer_original}")
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")

  # print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")
print(f"# CER PREDICTION: {float(total_cer_pred/NUM_SAMPLE)}")
print(f"# CER ORIGINAL: {float(total_cer_ori/NUM_SAMPLE)}")

  0%|          | 9/5000 [00:42<5:50:25,  4.21s/it]

err hunspell: ลงไปใน


  0%|          | 15/5000 [01:12<6:42:07,  4.84s/it]

err hunspell: ๕๐


  1%|          | 26/5000 [02:12<7:36:25,  5.51s/it]

...

err hunspell: เป๊กผลิตโชค


100%|█████████▉| 4999/5000 [6:46:56<00:04,  4.88s/it]

err hunspell: ก็จะมี


100%|██████████| 5000/5000 [6:47:00<00:00,  4.88s/it]

AVG ACC: 5.882352941176471e-06
AVG f1: 1.0256410256410256e-05
# HIGHER BLEU PREDICTION: 0
# LOWER BLEU PREDICTION: 4985
# EQUAL BLEU PREDICTION: 15
# GLEU PREDICTION: 0.5371540918455342
# GLEU ORIGINAL: 0.8660844114008937
# CER PREDICTION: 0.18518745386969365
# CER ORIGINAL: 0.03541724276517062


In [ ]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

FILE = "drive/MyDrive/AIBuilders/mlm/tagging_nova_75.pth"
tagging_model = BertModel()
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.7.attention.self.query.bias', 'roberta.encoder.layer.5.attention.self.key.weight', 'roberta.encoder.layer.4.output.dense.bias', 'roberta.encoder.layer.3.attention.self.query.bias', 'roberta.encoder.layer.9.output.dense.bias', 'roberta.encoder.layer.2.attention.self.key.weight', 'roberta.encoder.layer.8.attention.self.value.bias', 'roberta.encoder.layer.10.attention.output.dense.weight', 'roberta.encoder.layer.11.attention.self.key.bias', 'roberta.encoder.layer.4.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.2.intermediate.dense.bias', 'roberta.encoder.layer.4.attention.self.key.bias', 'roberta.encoder.layer.7.output.dense.weight', 'roberta.encoder.layer.8.intermediate.dense.bias', 'roberta.encoder.layer.7.output.dense.bias', 'roberta.encoder.layer.8.attent

RuntimeError: ignored

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def evaluate_one_text(model, sentence, mask, labels):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    input_id = torch.Tensor([sentence]).type(torch.int64)
    label_ids = []
    for i in sentence:
      if i == 1 or i == 5 or i == 6:
        label_ids.append(-100)
      else:
        label_ids.append(2)
    label_ids = torch.Tensor([label_ids]).type(torch.int64)
    mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0

for i in tqdm(range(NUM_SAMPLE)):
  text_id = ds_tag.iloc[i]['text']['input_ids'].squeeze(0).tolist()
  text_token = tokenizer.convert_ids_to_tokens(text_id)
  original_token = text_token.copy()
  text_token_len = len(text_token)
  original = [original_token]
  labels_id = ds_mlm.iloc[i]['labels']
  mask = ds_mlm.iloc[i]['attention_mask']
  labels_token = tokenizer.convert_ids_to_tokens(labels_id)
  references = [[labels_token]]

  i_f = evaluate_one_text(tagging_model, text_id, mask, labels_id)
  print(i_f)
  i_f_len = len(i_f)

  print(f"NO. {i}, {''.join(text_token)}")

  for j in range(i_f_len):
    text_token[j+1] = text_token[j+1].replace("▁", " ")
    if(i_f[j] == 'i'):    
      if langid.classify(text_token[j+1])[0] == 'th':
        try:
          corrected = spell(text_token[j+1],"th_TH")
        except:
          print(f"err hunspell: {text_token[j+1]}")
        else:
          if not corrected == "No Suggestions":
            print(f"{text_token[j+1]} => {corrected[0]}")
            text_token[j+1] = corrected[0]
  predictions = [text_token]
  final = "".join(text_token)
  final = final.replace("▁", " ")

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  print(len(text_token), len(labels_token))
  print(text_token)
  print(labels_token)
  labels_len = len(labels_token)
  for i in range(labels_len):
    if not text_token[i] == original_token[i]: #change
      denom += 1
      if text_token[i] == labels_token[i]:
        numer += 1
        TP += 1
      elif not text_token[i] == labels_token[i]:
        FP += 1
    elif text_token[i] == original_token[i]: #no change
      if text_token[i] == labels_token[i]:
        TN += 1
      elif not text_token[i] == labels_token[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)

  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1
  
  print(f"ACC: {acc}")
  print(f"f1: {f1}")
  print(f"ORIGINAL : {bleu_original}")
  print(f"PREDICTED: {bleu_prediction}")

  print("\n-----------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")